In [ ]:
# add root folder of the project to path
import sys
sys.path.insert(0, '..')

In [ ]:
# parameter settings
is_plot = True
is_export = True

landmarks_path = '../data/landmarks/refine_6kmh_braless_18markers_12fps.pkl'
meshes_path = '../data/meshes/6kmh_braless_26markers/'
test_landmarks_path = '../data/test/braless_random_landmarks.pkl'

start=0
stride = 12
end=120

export_folder = "output/data/"

# Data Loading

In [ ]:
from mesh4d import obj3d

mesh_ls, texture_ls = obj3d.load_mesh_series(
    folder=meshes_path,
    start=start,
    stride=stride,
    end=end,
)

In [ ]:
from mesh4d import utils

landmarks = utils.load_pkl_object(landmarks_path)
landmarks.interp_field()

In [ ]:
from mesh4d.analyse.crave import clip_with_contour

contour = landmarks.extract(('marker 0', 'marker 2', 'marker 3', 'marker 14', 'marker 15', 'marker 17'))
mesh_clip_ls = clip_with_contour(mesh_ls, start_time=0, fps=120/stride, contour=contour, clip_bound='xy', margin=30)

In [ ]:
body_ls = obj3d.init_obj_series(
    mesh_ls, 
    obj_type=obj3d.Obj3d_Deform
    )

In [ ]:
breast_ls = obj3d.init_obj_series(
    mesh_clip_ls, 
    obj_type=obj3d.Obj3d_Deform
    )

In [ ]:
from mesh4d import obj4d

body4d = obj4d.Obj4d_Deform(
    fps=120 / stride,
    enable_rigid=False,
    enable_nonrigid=False,
)
body4d.add_obj(*body_ls)
body4d.load_markerset('landmarks', landmarks)

In [ ]:
mesh_clip_ls[0]

In [ ]:
from mesh4d import obj4d

breast4d = obj4d.Obj4d_Deform(
    fps=120 / stride,
    enable_rigid=False,
    enable_nonrigid=False,
)
breast4d.add_obj(*breast_ls)
breast4d.load_markerset('landmarks', landmarks)

# Data Visualisation

In [ ]:
if is_export:
    body4d.gif_animate(output_folder=export_folder, filename='body4d')

In [ ]:
if is_plot:
    body4d.show(elements='m')

In [ ]:
if is_plot:
    body4d.show(elements='mk')

In [ ]:
if is_export:
    breast4d.gif_animate(output_folder=export_folder, filename='breast4d')
    breast4d.gif_animate(output_folder=export_folder, filename='breast4d_mesh', elements='m')

In [ ]:
if is_plot:
    import pyvista as pv

    stack_dist = 1000
    zoom_rate = 3.5
    window_size = [2000, 800]
    
    scene = pv.Plotter()
    plot_num = len(body4d.obj_ls)

    for idx in range(0, plot_num):
        body = body4d.obj_ls[idx]
        breast = breast4d.obj_ls[idx]
        width = body.get_width()

        body.add_mesh_to_scene(scene, location=[0, 0, idx*stack_dist], opacity=0.1)
        breast.add_mesh_to_scene(scene, location=[0, 0, idx*stack_dist])
        breast.add_kps_to_scene(scene, location=[0, 0, idx*stack_dist], radius=0.02*width)
        
    scene.camera_position = 'zy'
    scene.camera.azimuth = 45
    scene.camera.zoom(zoom_rate)
    scene.window_size = window_size
    scene.enable_parallel_projection()
    scene.show()

# Compared with nodes extracted from FE model

## Before alignment

In [ ]:
import pandas as pd

nodes = pd.read_excel("../data/simulation/node_position.xlsx")
points = nodes.to_numpy()

In [ ]:
import pyvista as pv

scene = pv.Plotter()
scene.add_points(points, point_size=1e-3)

breast_ls[0].add_mesh_to_scene(scene)

scene.show()

## Aligned with rigid CPD

In [ ]:
from probreg import cpd

tf_param, _, _ = cpd.registration_cpd(
    source=breast_ls[0].get_sample_points(1000), 
    target=points, 
    tf_type_name='rigid',
    )

In [ ]:
tf_inverse = tf_param.inverse()
tf_inverse.rot, tf_inverse.scale, tf_inverse.t

In [ ]:
import numpy as np

aligned_points = (tf_inverse.scale * np.matmul(tf_inverse.rot, points.T)).T + tf_inverse.t

## Surface distance after alignment

In [ ]:
import pyvista as pv

scene = pv.Plotter()
scene.add_points(points, point_size=1e-3)
scene.add_points(aligned_points, point_size=1e-3, color='gold')
body_ls[0].add_mesh_to_scene(scene, opacity=0.5)

scene.show()

In [ ]:
from mesh4d.analyse import measure

nearest_points = measure.nearest_points_from_plane(body_ls[0].mesh, aligned_points)

In [ ]:
import numpy as np

dist = np.linalg.norm(nearest_points - aligned_points, axis=1)
print("surface distance: {:.2f} ± {:.2f} (mm)".format(np.mean(dist), np.std(dist)))